In [1]:
import pandas as pd

def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_u'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_2_u'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_3_u'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['contents_attribute_h_u'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 상위코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

In [48]:
d_code = pd.read_csv('D_code.csv', index_col=0).T.to_dict()
h_code = pd.read_csv('H_code.csv', index_col=0).T.to_dict()
l_code = pd.read_csv('L_code.csv', index_col=0).T.to_dict()

df_train = pd.read_csv('train.csv', index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)

df_train = add_code(df_train, d_code, h_code, l_code)
df_test = add_code(df_test, d_code, h_code, l_code)

submission_df = pd.read_csv('sample_submission.csv')

In [49]:
df_train.drop(['person_prefer_g', 'person_prefer_f'], axis=1, inplace=True)
df_test.drop(['person_prefer_g', 'person_prefer_f'], axis=1, inplace=True)

df_train['contents_open_dt'] = pd.to_datetime(df_train['contents_open_dt'])
df_test['contents_open_dt'] = pd.to_datetime(df_test['contents_open_dt'])

In [51]:
def split_datetime(df) :
#     df['year'] = df['contents_open_dt'].dt.year
#     df['month'] = df['contents_open_dt'].dt.month
#     df['day'] = df['contents_open_dt'].dt.day
#     df['hour'] = df['contents_open_dt'].dt.hour
    df['dow'] = df['contents_open_dt'].dt.dayofweek
#     df['minute'] = df['contents_open_dt'].dt.minute
    return df

In [52]:
df_train = split_datetime(df_train)
df_test = split_datetime(df_test)

for i in ['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn', 'h_m_match_yn', 'h_s_match_yn'] :
    df_train[i] = df_train[i].astype('int')
    df_test[i] = df_test[i].astype('int')

In [68]:
df_train_x = df_train.drop(['contents_open_dt', 'target'], axis=1)
df_train_y = df_train[['target']]

df_test_x = df_test.drop(['contents_open_dt'], axis=1)

from sklearn.model_selection import train_test_split

new_train_x, new_val_x, new_train_y, new_val_y = train_test_split(df_train_x, df_train_y, test_size=0.4 ,random_state=777)
new_val_x, new_test_val_x, new_val_y, new_test_val_y = train_test_split(new_val_x, new_val_y, test_size=0.5,random_state=777)

In [69]:
print(new_train_x.shape[0] / 500000)
print(new_val_x.shape[0] / 500000)
print(new_test_val_x.shape[0] / 500000)

0.60234
0.20078
0.200782


In [77]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

params = {
    'learning_rate' : 0.05,
        'max_depth' : 16,
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'is_training_metric' : True,
    'num_leaves' : 512,
#     'min_data_in_leaf' : 100,  # 100~1000
    'feature_fraction' : 0.7,
    'bagging_fraction' : 0.8,
    'bagging_freq':5,
    'seed':2020
}

train_dataset = lgb.Dataset(new_train_x, label=new_train_y)
val_dataset = lgb.Dataset(new_val_x, label=new_val_y)

model = lgb.train(params, train_dataset, 2000, val_dataset, verbose_eval=200, early_stopping_rounds=100)

pred_y = model.predict(new_val_x)
pred_y = [1 if i>0.5 else 0 for i in pred_y]
print('val acc : ',accuracy_score(new_val_y, pred_y))

pred_y = model.predict(new_test_val_x)
pred_y = [1 if i>0.5 else 0 for i in pred_y]
print('test val acc : ',accuracy_score(new_test_val_y, pred_y))
# 0.6171755674498294
# 0.6222225173654922
# 0.6269440718260662 -> 0.64806  // 2021-12-23 14:54
# 0.6292749658002736 

# 0.6118177770033494

# 0.6306296734092147  -> 2021-12-28 15:25:05	0.644898965

# 0.6308421765635583

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 150249, number of negative: 150921
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.165955 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4850
[LightGBM] [Info] Number of data points in the train set: 301170, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498884 -> initscore=-0.004463
[LightGBM] [Info] Start training from score -0.004463
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's binary_logloss: 0.6387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's binary_logloss: 0.6

In [24]:
pred_y = model.predict(df_test_x)
pred_y = [1 if i>0.5 else 0 for i in pred_y]
submission_df['target'] = pred_y
submission_df['target'].value_counts()
submission_df.to_csv('sub15.csv', index=False)

In [75]:
from sklearn.naive_bayes import CategoricalNB, MultinomialNB, GaussianNB

gnb1 = CategoricalNB()
gnb2 = MultinomialNB()
gnb3 = GaussianNB()

gnb1.fit(new_train_x, new_train_y)
gnb2.fit(new_train_x, new_train_y)
gnb3.fit(new_train_x, new_train_y)

pred_y = gnb1.predict(new_val_x)
print(accuracy_score(pred_y, new_val_y))
pred_y = gnb2.predict(new_val_x)
print(accuracy_score(pred_y, new_val_y))
pred_y = gnb3.predict(new_val_x)
print(accuracy_score(pred_y, new_val_y))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.6009861539994024
0.5171630640502042
0.5346448849487001


In [ ]:
0.6009662316963841
0.6142444466580337
0.6170136467775675